In [ ]:
#%%capture
%pip install --upgrade pyyaml pyvelociraptor grpcio grpcio-tools cryptography

In [ ]:
import yaml
import grpc
import pyvelociraptor
from pyvelociraptor import api_pb2
from pyvelociraptor import api_pb2_grpc

config_path = 'api.config.yaml'
org_id = '<root>'
env = []

query = """
SELECT * from info()
"""

In [ ]:
#Read the config file
config = pyvelociraptor.LoadConfigFile(config_path)

# Fill in the SSL params from the api_client config file. You can get such a file:
# velociraptor --config server.config.yaml config api_client > api_client.conf.yaml
creds = grpc.ssl_channel_credentials(
     root_certificates=config["ca_certificate"].encode("utf8"),
     private_key=config["client_private_key"].encode("utf8"),
     certificate_chain=config["client_cert"].encode("utf8"))

options = (('grpc.ssl_target_name_override', "VelociraptorServer",),)

# The first step is to open a gRPC channel to the server..
with grpc.secure_channel(config["api_connection_string"],creds, options) as channel:
     stub = api_pb2_grpc.APIStub(channel)

# The request consists of one or more VQL queries. Note that
# you can collect artifacts by simply naming them using the
# "Artifact" plugin.
request = api_pb2.VQLCollectorArgs(
          org_id=org_id,
          max_wait=1,
          max_row=100,
          #timeout=timeout,
          Query=[api_pb2.VQLRequest(
               Name="Test",
               VQL=query,
          )],
          env=env,
     )
# This will block as responses are streamed from the
# server. If the query is an event query we will run this loop
# forever.

for response in stub.Query(request):
     if response.Response:
          # Each response represents a list of rows. The columns
          # are provided in their own field as an array, to
          # ensure column order is preserved if required. If you
          # dont care about column order just ignore the Columns
          # field. Note that although JSON does not specify the
          # order of keys in a dict Velociraptor always
          # maintains this order so an alternative to the
          # Columns field is to use a JSON parser that preserves
          # field ordering.

          # print("Columns %s:" % response.Columns)

          # The actual payload is a list of dicts. Each dict has
          # column names as keys and arbitrary (possibly nested)
          # values.
          package = json.loads(response.Response)
          print (package)

     elif response.log:
          # Query execution logs are sent in their own messages.
          print ("%s: %s" % (time.ctime(response.timestamp / 1000000), response.log))
